In [23]:
# Generating a toy dataset.
# DO NOT MODIFY THIS PART

import numpy as np
import math
import random as rand

paras = list((rand.random() - 0.5 for _ in range(9)))

def y_gen(x):
    h_11 = math.tanh(paras[0] * x[0] + paras[1] * x[1] + paras[2])
    h_12 = math.tanh(paras[3] * x[0] + paras[4] * x[1] + paras[5])
    h_21 = 1/(1 + np.exp(-(paras[6] * h_11 + paras[7] * h_12 + paras[8])))
    return h_21 + ((rand.random()-0.5)/100 if rand.random()>0.6 else 0)

n = 300
x = list(zip((rand.random() - 0.5 for _ in range(n)), (rand.random() - 0.5 for _ in range(n))))
y = list(map(y_gen, x))
y = [(i-min(y))/(max(y)-min(y)) for i in y]

In [24]:
# Spliting dataset into training, validation, and test.
# DO NOT MODIFY THIS PART

from sklearn.model_selection import train_test_split

r = 0.25
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=r)

x_val = x_train[:int(n*r)]
partial_x_train = x_train[int(n*r):]
y_val = y_train[:int(n*r)]
partial_y_train = y_train[int(n*r):]

In [25]:
# DO NOT MODIFY THIS PART

from keras import models
from keras import layers
from keras import optimizers

def create_model(learning_rate=0.01, beta_1=0.9, beta_2=0.999):
    model = models.Sequential()
    model.add(layers.Dense(2, activation='tanh', input_shape=(2,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2), loss='mse')
    return model

In [26]:
# Complete the cell to find a best hyper-parameter setting using GridSerachCV

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

unoptimized_model = KerasRegressor(build_fn=create_model, epochs=60, verbose=0)

hyperparameters_dictionary = {'learning_rate': [0.005, 0.007, 0.009, 0.01, 0.03, 0.05],  
                              'beta_1': [0.93, 0.9, 0.87, 0.84, 0.81, 0.78], 
                              'beta_2': [0.999, 0.990,0.989,0.980,0.979,0.970] }

optimized_model = GridSearchCV(estimator=unoptimized_model, param_grid=hyperparameters_dictionary,n_jobs=-1, cv=3)

optimized_hyperparameters = optimized_model.fit(partial_x_train,partial_y_train, validation_data=(x_val, y_val))


/var/folders/yq/7r63wqd560v9hsh_n2dfg0m40000gn/T/ipykernel_3644/2853210189.py:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  unoptimized_model = KerasRegressor(build_fn=create_model, epochs=60, verbose=0)


In [27]:

#print(optimized_hyperparameters.cv_results_)

mse = optimized_hyperparameters.cv_results_['mean_test_score']
combinations = optimized_hyperparameters.cv_results_['params']
for error, combination in zip(mse, combinations):
    print(error, combination)
    
print("Best: ",optimized_hyperparameters.best_score_, optimized_hyperparameters.best_params_)

-0.00478614695991079 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.005}
-0.025453781243413687 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.007}
-0.004777221474796534 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.009}
-0.00442639971151948 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.01}
-0.004409069272999962 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.03}
-0.004528295403967301 {'beta_1': 0.93, 'beta_2': 0.999, 'learning_rate': 0.05}
-0.007802768299976985 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.005}
-0.004942186487217744 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.007}
-0.005368590432529648 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.009}
-0.004381504530707995 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.01}
-0.004835960688069463 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.03}
-0.004371715942397714 {'beta_1': 0.93, 'beta_2': 0.99, 'learning_rate': 0.05}
-0.008516601907710234 {'beta_1': 0.93, 'beta_2': 0.989